<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-LLM-ZeroToAll/02_prompt_engineering.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
! pip3 install -qU langchain-upstage

In [2]:

%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Quick hello world
from langchain_upstage import ChatUpstage

llm = ChatUpstage()
llm.invoke("What's the best season to get to Korea?")

AIMessage(content="The best season to visit Korea depends on your personal preferences and what you want to do while you're there.\n\nIf you're interested in seeing cherry blossoms, the best time to visit is in the spring, usually in late March to early April. This is also a great time to visit if you want to enjoy mild weather and see the country in bloom.\n\nSummer, from June to August, is the peak tourist season in Korea, and the weather is generally warm and humid. This is a great time to visit the beaches and enjoy outdoor activities, but it can also be very crowded.\n\nFall, from September to November, is another popular time to visit Korea. The weather is typically cool and crisp, and the foliage is beautiful. This is also a great time to visit if you want to experience traditional Korean festivals and events.\n\nWinter, from December to February, can be very cold, especially in the mountainous regions. However, it can also be a beautiful time to visit, with snow-covered landsca

In [5]:
# Chat prompt
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "What is the capital of France?"),
        ("ai", "I know of it. It's Paris!!"),
        ("human", "What about Korea?"),
    ]
)

# 3. define chain
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser()
chain.invoke({})



"Oh, that's a good one! The capital of Korea is Seoul!"

<font color='orange'>[Comment of sung kim]</font>

- 내가 context를 만드는 거: few-shot learning. 위의 예시처럼 `PromptTemplate` 같은 걸 사용
- 또는 대화의 history를 넘겨줄수도 있음.
- 위에 `chain`이 있죠? language의 chain이라서 langchain이라고 부름. 옛날식으로 말하면 pipeline.
위의 코드는 `chat_prompt`를 llm에 넣고 그 결과를 parser에 넣는다는 뜻.
simple해서 좋음!

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The cafeteria started with 23 apples. After using 20 to make lunch, they would have:\n\n23 - 20 = 3 apples left.\n\nThen, they bought 6 more apples, so they would have:\n\n3 + 6 = 9 apples.\n\nTherefore, the cafeteria has 9 apples.'

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The answer is 29.'

![CoT](figures/cot.webp)

from https://arxiv.org/abs/2201.11903

<font color='orange'>[Comment of Sung Kim]</font>
- Chain of Thought. CoT
- 이게 왜 될까요? llm은 뒷말잇기를 하는 아이이기 때문에 좀 더 말이 되는 쪽으로 뒷말잇기를 하기 위해서 성능이 올라가는 게 아닐까요?

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The cafeteria had 23 apples. They used 20, so they had 3 left. Then they bought 6 more, so they had 9. The answer is 9.'

![Zero-Shot COT](figures/zero-cot.webp)

From https://arxiv.org/abs/2205.11916

<font color='orange'>[Comment of Sung Kim]</font>
- 예제를 안 주고 step-by-step으로 생각하라고 말만 해도 비슷한 효과.

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: Let's think step by step.
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'Step 1: The cafeteria started with 23 apples.\nStep 2: They used 20 apples to make lunch, so we subtract 20 from 23.\nStep 3: The number of apples left after using some is 3.\nStep 4: They bought 6 more apples, so we add 6 to 3.\nStep 5: The final number of apples is 9.\n\nA: The cafeteria has 9 apples now.'

## divide and conquer
![divideandconquer](figures/divideandconquer.png)

<font color='orange'>[Comment of Sung Kim]</font>
한번에 많이 시키지 말고 나눠서 시키자

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide three questions from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'1. What is SOLAR 10.7B and what are its key features?\n2. What is depth up-scaling (DUS) and how does it differ from other LLM up-scaling methods?\n3. How does SOLAR 10.7B-Instruct compare to Mixtral-8x7B-Instruct in terms of instruction-following capabilities?'

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please extract three keywords from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'1. Large Language Model (LLM)\n2. Depth Up-Scaling (DUS)\n3. Instruction-Following Capabilities'

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "Depth up-scaling (DUS)":
    
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'What is the main difference between depth up-scaling (DUS) and other LLM up-scaling methods that use mixture-of-experts?'

<font color='orange'>[Comment of Sung Kim]</font>
- 질문 3개 뽑아와 vs. [키워드 3개 뽑아와 -> 그거에 대한 질문을 뽑아] 같은 2-step

<font color='orange'>[Comment of Sung Kim]</font>

variable 활용해봅시다

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [14]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"adjective": "funny", "content": "chickens"})

"Why don't chickens have any friends?\n\nBecause they always run away when they see other animals coming!"

<font color='orange'>[Comment of Sung Kim]</font>
- 파이프라인 구축 후에 variable에 값 넣기.
- 이렇게 하면 좋은 점: 파이프라인 재사용 가능

In [15]:
chain.invoke({"adjective": "funny", "content": "beef"})

"Why did the beef cross the road?\n\nTo get to the other side.\n\nBut seriously, beef doesn't cross roads. It's usually the other way around. People cross roads to get to the beef."

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "{keyword}":
    
    ---
    {text}
    """
)
chain = prompt_template | llm | StrOutputParser()
keyword = "DUS"
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
"""
chain.invoke({"keyword": keyword, "text": text})

'What is the difference between the DUS method for scaling LLMs and other up-scaling methods that use mixture-of-experts?'

# Excercise 

Complete this code to create excellent questions from given documents using the following steps:
1. Include Few Shot examples, including CoT.
2. Generate keywords and topics first.
3. Iterate through each keyword and topic to generate questions.
4. Compare the generated questions with zero-shot generated questions.


# References
* https://platform.openai.com/docs/guides/prompt-engineering 
* https://docs.anthropic.com/claude/docs/intro-to-prompting 
* https://smith.langchain.com/hub 